In [22]:
import fastf1 as ff1
from fastf1 import plotting
import numpy as np
import plotly.graph_objects as go
import random

In [23]:
# Enable the cache
ff1.Cache.enable_cache('cache') 

# Setup plotting
plotting.setup_mpl()

c:\Users\Vitrag Khatadia\AppData\Local\Programs\Python\Python39\lib\site-packages\fastf1\plotting\_plotting.py:90: FutureWarning:

FastF1 will no longer silently modify the default Matplotlib colors in the future.
To remove this warning, explicitly set `color_scheme=None` or `color_scheme='fastf1'` when calling `.setup_mpl()`.

c:\Users\Vitrag Khatadia\AppData\Local\Programs\Python\Python39\lib\site-packages\fastf1\plotting\_plotting.py:98: FutureWarning:

FastF1 will stop modifying the default Matplotlib settings in the future.
To opt-in to the new behaviour and remove this warning, explicitly set `misc_mpl_mods=False` when calling `.setup_mpl()`.



In [24]:
# Load the session data
quali = ff1.get_session(2024, 'Silverstone', 'Q')

In [25]:
quali

2024 Season Round 12: British Grand Prix - Qualifying

In [26]:
quali.load()

core           INFO 	Loading data for British Grand Prix - Qualifying [v3.4.0]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['63', '44', '4', '1', '81', '27', '55', '18', '23', '14', '16', '2', '22', '24', '3', '77', '20', '31', '11', '10']


In [27]:
q1,q2,q3=quali.laps.split_qualifying_sessions()

In [28]:
# Get the telemetry data for the fastest lap of the three drivers
ver_lap = q3.pick_driver('VER').pick_fastest()
ham_lap = q3.pick_driver('HAM').pick_fastest()
nor_lap = q3.pick_driver('NOR').pick_fastest()

In [29]:
# Get the telemetry data and add distance
ver_tel = ver_lap.get_telemetry().add_distance()
ham_tel = ham_lap.get_telemetry().add_distance()
nor_tel = nor_lap.get_telemetry().add_distance()

core        WARNING 	Failed to preserve data type for column 'X' while merging telemetry.
core        WARNING 	Failed to preserve data type for column 'Y' while merging telemetry.
core        WARNING 	Failed to preserve data type for column 'Z' while merging telemetry.
core        WARNING 	Failed to preserve data type for column 'X' while merging telemetry.
core        WARNING 	Failed to preserve data type for column 'Y' while merging telemetry.
core        WARNING 	Failed to preserve data type for column 'Z' while merging telemetry.


In [30]:
ver_tel['X'] = ver_tel['X'].astype(float)
ver_tel['Y'] = ver_tel['Y'].astype(float)
ver_tel['Z'] = ver_tel['Z'].astype(float)

nor_tel['X'] = nor_tel['X'].astype(float)
nor_tel['Y'] = nor_tel['Y'].astype(float)
nor_tel['Z'] = nor_tel['Z'].astype(float)

ham_tel['X'] = ham_tel['X'].astype(float)
ham_tel['Y'] = ham_tel['Y'].astype(float)
ham_tel['Z'] = ham_tel['Z'].astype(float)

In [31]:
minisectors = np.linspace(0, max(ver_tel['Distance'].max(), ham_tel['Distance'].max(), nor_tel['Distance'].max()), 25)

In [32]:
# Create an array to store which driver was fastest in each minisector
minisector_winner = []
sector_winner_count = {'VER': 0, 'HAM': 0, 'NOR': 0}

In [33]:
# Determine which driver was fastest in each minisector
for i in range(len(minisectors) - 1):
    ver_speed = ver_tel[(ver_tel['Distance'] >= minisectors[i]) & (ver_tel['Distance'] < minisectors[i+1])]['Speed'].mean()
    ham_speed = ham_tel[(ham_tel['Distance'] >= minisectors[i]) & (ham_tel['Distance'] < minisectors[i+1])]['Speed'].mean()
    nor_speed = nor_tel[(nor_tel['Distance'] >= minisectors[i]) & (nor_tel['Distance'] < minisectors[i+1])]['Speed'].mean()
    
    if nor_speed > ver_speed and nor_speed > ham_speed:
        minisector_winner.append('NOR')
        sector_winner_count['NOR'] += 1
    elif ver_speed > ham_speed:
        minisector_winner.append('VER')
        sector_winner_count['VER'] += 1
    else:
        minisector_winner.append('HAM')
        sector_winner_count['HAM'] += 1


In [37]:
# Create Plotly figure
fig = go.Figure()

# Function to generate a random color
def random_color():
    return "#"+''.join([random.choice('0123456789ABCDEF') for _ in range(6)])

# List of driver keys
driver_keys = ['VER', 'HAM', 'NOR', 'RUS', 'LEC', 'SAI', 'ALO', 'PER', 'BOT', 'OCO', 
               'GAS', 'TSU', 'ZHO', 'MAG', 'HUL', 'ALB', 'SAR', 'LAW', 'PIA', 'STR']

# Dynamically assign a random color to each driver
colors = {driver: random_color() for driver in driver_keys}

# Add trace for each driver
for driver in ['VER', 'HAM', 'NOR']:
    fig.add_trace(go.Scatter(x=[None], y=[None], mode='lines', 
                             line=dict(color=colors[driver], width=4), 
                             name=driver, legendgroup=driver))

# Plot each minisector
for i in range(len(minisectors) - 1):
    winner = minisector_winner[i]
    x = ver_tel[(ver_tel['Distance'] >= minisectors[i]) & (ver_tel['Distance'] < minisectors[i+1])]['X'].values
    y = ver_tel[(ver_tel['Distance'] >= minisectors[i]) & (ver_tel['Distance'] < minisectors[i+1])]['Y'].values
    
    fig.add_trace(go.Scatter(
        x=x, y=y,
        mode='lines',
        line=dict(color=colors[winner], width=6, dash='solid'),
        name=winner,
        showlegend=False
    ))

# Determine the fastest driver overall
fastest_driver = q3.pick_fastest().Driver
fastest_time = q3.pick_fastest().LapTime

# Beautify the plot
fig.update_layout(
    title={
        'text': f'Track Dominance Chart - {quali} 3',
        'x': 0.5,
        'xanchor': 'center',
        'font': {'size': 20, 'color': 'white'}
    },
    xaxis_title='Distance (m)',
    yaxis_title='Track Position',
    legend_title='Drivers',
    plot_bgcolor='black',
    paper_bgcolor='black',
    xaxis=dict(visible=False),
    yaxis=dict(visible=False),
    font=dict(family='Arial', size=14, color='white'),
    annotations=[
        dict(
            x=0.5, y=1.05, xref='paper', yref='paper',
            text=f"Fastest Driver: {fastest_driver} ({fastest_time})",
            showarrow=False,
            font=dict(size=16, color=colors[fastest_driver])
        )
    ],
    margin=dict(l=0, r=0, t=100, b=0)  # Adjust margins to fit title and annotations
)

fig.show()
